In [2]:
#################
# Miscellaneous #
#################
'''
def is_fundamental_discriminant(D):
    Returns true if '-D' is a fundamental discriminant.
'''

#
# Returns true if '-D' is a fundamental discriminant.
#
def is_fundamental_discriminant(D):
    discriminant = Integer(-D)
    if discriminant % 4 == 0:
        d = Integer(discriminant/4)
        return d % 4 != 1 and is_squarefree(d)
    if discriminant % 4 == 1:
        return is_squarefree(discriminant)
    return False

In [3]:
###################
# Quadratic Forms #
###################
'''
def QuadraticForm:
    Represents a quadratic form constructed from 'a', 'b', and 'c'.
    Quadratic forms are automatically reduced upon creation.
    
    def __add__(f, g):
        Returns the Gaussian composition of 'f' and 'g'.
    def __mul__(f, n):
        Returns the 'n'th composition of 'f' with itself.
    def __neg__(f):
        Returns the inverse of 'f'.
    def order(f):
        Returns the order of a form in the class group.
    def discriminant(f):
        Returns the discriminant of a form.
    def a(f), b(f), c(f):
        Return the coefficients of a form.

def principal_form(D):
    Returns the principal form of discriminant '-D', which corresponds to identity in the class group.
'''

#
# Represents a primitive positive definite binary quadratic form.
#
class QuadraticForm:
    #
    # An instance of the 'QuadraticForm' class is constructed using the syntax 'QuadraticForm(a,b,c)',
    # where 'a', 'b', and 'c' are the desired coefficients for the form. If the quadratic form is not
    # primitive or is not positive definite, the constructor will print an error message. Each form
    # is reduced immediately upon construction, so all instances of the 'QuadraticForm' class are
    # guaranteed to be in reduced form and may be compared freely.
    #
    def __init__(f, a, b, c):
        # Check that the form is primitive.
        if gcd([a, b, c]) != 1:
            print("The quadratic form '" + str((a,b,c)) + "' is not primitive!")
            return
        # Check that the form is positive definite.
        if b^2-4*a*c >= 0:
            print("The quadratic form '" + str((a,b,c)) + "' is not positive definite!")
            return
        # Initialize the coefficient members, then reduce the form.
        f._a = Integer(a); f._b = Integer(b); f._c = Integer(c)
        f._reduce()
    #
    # Quadratic forms are represented as tuples for brevity and utility.
    #
    def __repr__(f):
        return str((f._a, f._b, f._c))
    #
    # Returns true if 'f' is equal to 'g'. Because all forms are automatically reduced, this can be
    # used to test whether two forms are properly equivalent.
    #
    def __eq__(f, g):
        return f._a == g._a and f._b == g._b and f._c == g._c
    #
    # Returns the composition of 'f' and 'g'. If 'f' and 'g' do not have the same discriminant, then
    # this method prints an error message.
    #
    def __add__(f, g):
        # Find the discriminant, and check that it is common between 'f' and 'g'.
        D = -f.discriminant()
        if D != -g.discriminant():
            print("The forms '" + str(f) + "' and '" + str(g) + "' have different discriminants, and so cannot be composed!")
            return
        # Implement Arndt's method for computing the composition of 'f' and 'g'.
        b = Integer((f._b + g._b)/2)
        e = gcd([b, f._a, g._a])
        A = Integer(f._a*g._a/e^2)
        G, h, k = xgcd(Integer(g._a/e), Integer(f._a/e))
        x = Integer(h*f._b*g._a/e) + Integer(k*f._a*g._b/e)
        _, m, n = xgcd(G, Integer(b/e))
        B = (m*x + n*Integer((f._b*g._b-D)/(2*e))) % (2*A)
        # Return the computed composition.
        return QuadraticForm(A, B, Integer((B^2+D)/(4*A)))
    def __sub__(f, g):
        return f + -g
    #
    # Returns the 'n'th composition of 'f' with itself.
    #
    def __mul__(f, n):
        if n < 0:
            return (-f)*(-n)
        if n == 0:
            return principal_form(-f.discriminant())
        m = 1; g = f
        while 2*m <= n:
            m *= 2; g += g
        return g + (n-m)*f
    def __rmul__(f, n):
        return f*n
    #
    # Returns the inverse of 'f'.
    #
    def __neg__(f):
        return QuadraticForm(f._a, -f._b, f._c)
    def __pos__(f):
        return f
    #
    # An internal function used to reduce a form upon creation.
    #
    def _reduce(f):
        # Repeat the reduction algorithm until the form is reduced.
        while True:
            # Apply an integer translation to the root of the form.
            n = (f._b+f._a-1)//(2*f._a)
            f._c += f._a*n^2 - f._b*n; f._b -= 2*f._a*n
            # If the form is reduced, we are done.
            if f._a < f._c or (f._a <= f._c and f._b >= 0):
                break
            # Otherwise, apply an inversion transformation to the form.
            f._a, f._c = f._c, f._a
            f._b = -f._b
        # At this point, the form is necessarily reduced.
        return
    #
    # Returns the order of a quadratic form.
    #
    def order(f):
        g = f; n = 1; D = -f.discriminant()
        while g != principal_form(D):
            g += f; n += 1
        return n
    #
    # Returns the discriminant of this form, which is necessarily a negative number congruent to
    # zero or one mod four.
    #
    def discriminant(f):
        return f._b^2-4*f._a*f._c
    #
    # Getter functions for the coefficients 'a', 'b', and 'c' of a form.
    #
    def a(f):
        return f._a
    def b(f):
        return f._b
    def c(f):
        return f._c

#
# Returns the principal form of discriminant '-D'. If 'D' is not positive or not a valid discriminant,
# then this method prints an error message.
#
def principal_form(D):
    if D <= 0:
        print("The number '" + str(-D) + "' is not the discriminant of a positive definite quadratic form!")
        return
    if D % 4 == 0:
        return QuadraticForm(1, 0, Integer(D/4))
    if (D+1) % 4 == 0:
        return QuadraticForm(1, 1, Integer((D+1)/4))
    print("The number '" + str(-D) + "' is not a valid discriminant!")
    return

In [4]:
################
# Class Groups #
################
'''
class ClassGroup:
    Represents a complete class group of a particular discriminant.
    All reduced forms of the class group are computed upon its creation.
    
    def discriminant(G):
        Returns the discriminant of a class group.
    def order(G):
        Returns the order of a class group.
    def forms(G):
        Returns the complete list of reduced forms contained in a class group.
'''

#
# Represents a class group, consisting of all of the primitive positive definite binary quadratic forms
# of a given discriminant. This class automatically gathers and stores information about the group,
# including its order and structure as an abelian group.
#
class ClassGroup:
    #
    # An instance of the 'ClassGroup' class corresponding to quadratic forms of discriminant '-D' is
    # constructed using the syntax 'ClassGroup(D)'.
    #
    def __init__(G, D):
        # Check that '-D' is a valid negative discriminant, and store it as a member.
        if D <= 0:
            print("The number '" + str(-D) + "' is not the discriminant of a positive definite quadratic form!")
            return
        if (D+3) % 4 == 0 or (D+2) % 4 == 0:
            print("The number '" + str(-D) + "' is not a valid discriminant!")
            return
        G._discriminant = Integer(-D)
        # Compute the forms of the class group.
        G._compute_forms()
    #
    # Class groups are represented by a list of their elements.
    #
    def __repr__(G):
        return str(G._forms)
    #
    # Computes the representatives of the forms in a class group and stores them as a member.
    #
    def _compute_forms(G):
        G._forms = []; D = -G._discriminant
        # Enumerate through all possible values of 'a'.
        a = 1
        while 3*a^2 <= D:
            # Enumerate through all values of 'b' from 'a' down to '1-a'. This ensures that the root
            # of each form lies in the strip 'a <= imag(z) < a'.
            b = a
            while b > -a:
                # Check that the root of the form satisfies 'abs(z) > 1' or 'abs(z) == 1' and 'b >= 0'.
                # This ensures that the root of the form lies in the fundamental domain.
                if (b >= 0 and b^2+D >= 4*a^2) or b^2+D > 4*a^2:
                    # Check that the necessary value for 'c' is integral.
                    if (b^2+D) % (4*a) == 0:
                        c = Integer((b^2+D)/(4*a))
                        # If the resulting form is primitive, store it.
                        if gcd([a,b,c]) == 1:
                            G._forms += [QuadraticForm(a,b,c)]
                b -= 1
            a += 1
        return
    #
    # Getter functions for the discriminant and the forms of a class group.
    #
    def discriminant(G):
        return G._discriminant
    def order(G):
        return len(G._forms)
    def forms(G):
        return G._forms

In [5]:
###################
# Elliptic Curves #
###################
'''
def get_de_t(E, t)
def find_good_t(E)
def P_Q_on_curves(E, D, P, Q)
def get_griffin_ono_discriminant(a4, a6, t):
    Returns the absolute value of the discriminant 'D_E(t)' used in the Griffin-Ono paper.
def get_elliptic_curve_triple(x, y):
    Computes the minimal integers 'a', 'b', and 'c' such that '(x, y) == (a/c^2, b/c^3)'.
'''

def get_de_t(E, t):
    '''
    Input
        E: elliptic curve
        t: int >= 0
    Output D_E(t) > 0 (the negative discr is -D)
    '''
    a4 = E.a4()
    a6 = E.a6()
    return 4*(t**3+a4*t-a6)

#Compute values of t that give usable D_E(t)
def find_good_t(E):
    '''
    input: elliptic curve E
    output: 
        good_t: set of t < 13 that give fundamental discriminant
    '''
    good_t = set()
    for t in range(1,13):
        discr = get_de_t(E,t)
        if discr > 0 and is_fundamental_discriminant(discr):
            good_t.add(t)
    return good_t


def P_Q_on_curves(E, D, P, Q):
    '''
    Input:
        E: elliptic curve object
        D: discriminant
        P: point that should be on E
        Q: point that should be on E_{-D}
    Output:
        message that tells you whether P and Q are on the curves E and E_[-D]
    '''
    if P[1]^2 != P[0]^3 + E.a4()*P[0] + E.a6():
        return "P is not on E."
    if (-D * (Q[1]^2/4) != Q[0]^3 + E.a4()*Q[0] + E.a6()):
        return "Q is not on E_{-D}."
    return "Both P and Q are on the right curves!"

#
# Returns the absolute value of the discriminant 'D_E(t)' used in the Griffin-Ono paper.
#
def get_griffin_ono_discriminant(a4, a6, t):
    return 4*(t^3+a4*t-a6)

#
# Computes the minimal integers 'a', 'b', and 'c' such that '(x, y) == (a/c^2, b/c^3)'.
#
def get_elliptic_curve_triple(x, y):
    x_num = Rational(x).numerator(); y_num = Rational(y).numerator()
    x_denom = Rational(x).denominator(); y_denom = Rational(y).denominator()
    c = lcm(x_denom, y_denom); a = Integer(x*c^2); b = Integer(y*c^3)
    v = factor(gcd([c^6, a^3, b^2]))
    for q in v:
        p = q[0]; k = q[1]//6
        c = Integer(c/p^k); a = Integer(a/p^(2*k)); b = Integer(b/p^(3*k))
    return (a,b,c)

#
# Computes the pairing from the Griffin-Ono paper using a slightly modified algorithm that ensures 'l' is even
# whenever 'alpha/G' is even.
#
def get_griffin_ono_pseudo_pairing(P, Q, D):
    if P[2] == 0:
        print("The point 'P' is infinite!")
        return
    if Q[2] == 0:
        print("The point 'Q' is infinite!")
        return
    if D <= 0:
        print("The value '" + str(-D) + "' is not the discriminant of a positive definite binary quadratic form!")
        return
    if D % 4 == 1 or D % 4 == 2:
        print("The value '" + str(D) + " is not a valid discriminant!")
        return
    if not is_fundamental_discriminant(D):
        print("The discriminant '" + str(-D) + "' is not fundamental!")
        return
    a,b,c = get_elliptic_curve_triple(P[0], P[1]); u,v,w = get_elliptic_curve_triple(Q[0], Q[1])
    if v == 0:
        print("One of the coordinates of the point 'Q = " + str(Q) + "' is zero!")
        return
    if D % 2 == 1 and v % 2 == 1:
        print("The inputs 'D = " + str(D) + "' and 'v = " + str(v) + "' are both odd!")
        return
    A = a*w^2; B = b*w^3; U = c^2*u; V = c^3*v; dV2 = Integer(V^2*D/4)
    alpha = abs(A-U); G = gcd(alpha, V^2)
    if (B^2 + dV2) % alpha != 0:
        print("Something is wrong: 'alpha = " + str(alpha) + "' does not divide 'B^2 + V^2*(D/4) = " + str(B^2 + dV2) + "'!")
        return
    if (4*B^2) % G != 0:
        print("Something is wrong: 'G = " + str(G) + "' does not divide '4*B^2 = " + str(4*B^2) + "'!")
        return
    H = gcd(2*B, V)
    if H^2 % G != 0:
        print("Something is wrong: 'G = " + str(G) + "' does not divide 'H^2 = " + str(H^2) + "'!")
        return
    if Integer(V^2/G) % Integer(V/H) != 0:
        print("Something is wrong: 'V/H = " + str(Integer(V/H)) + "' does not divide 'V^2/G = " + str(Integer(V^2/G)) + "'!")
        return
    if gcd(Integer(V/H), Integer(alpha/G)) != 1:
        print("Something is wrong: 'V/H = " + str(Integer(V/H)) + "' and 'alpha/G = " + str(Integer(alpha/G)) + "' are not coprime!")
        return
    if Integer(alpha/G) % 2 == 1:
        k = (-inverse_mod(Integer(alpha/G), Integer(2*V/H))*Integer((2*B+V*D)/H)) % Integer(2*V/H)
        if (Integer(alpha/G)*k) % Integer(2*V/H) != -Integer((2*B+V*D)/H) % Integer(2*V/H):
            print("Something is wrong: '(alpha/G)*k = " + str(Integer(alpha/G)*k) + "' is not congruent to '-(2*B+V*D)/H = " + str(-Integer((2*B+V*D)/H)) + "' modulo '2*V/H = " + str(Integer(2*V/H)) + "'!")
            return
    if Integer(alpha/G) % 2 == 0:
        if (2*B+V*D) % 2 != 0:
            print("Something is wrong: 'alpha/G = " + str(Integer(alpha/G)) + "' is even, but '2*B+V*D = " + str(2*B+V*D) + "' is odd!")
            return
        k = (-inverse_mod(Integer(alpha/(2*G)), Integer(V/H))*Integer((2*B+V*D)/(2*H))) % Integer(V/H)
        if k % 2 == 1:
            k += Integer(V/H)
        if (Integer(alpha/G)*k) % Integer(2*V/H) != -Integer((2*B+V*D)/H) % Integer(2*V/H):
            print("Something is wrong: '(alpha/G)*k = " + str(Integer(alpha/G)*k) + "' is not congruent to '-(2*B+V*D)/H = " + str(-Integer((2*B+V*D)/H)) + "' modulo '2*V/H = " + str(Integer(2*V/H)) + "'!")
            return
    l = H*k; beta = Integer((2*B+l*Integer(alpha/G))/V)
    if (beta+D) % 2 != 0:
        print("Something is wrong: 'beta = " + str(beta) + "' does not have the same pairity as 'D = " + str(D) + "'!")
        return
    gamma = Integer(beta^2+D)
    if gamma % Integer(4*alpha/G) != 0:
        print("Something is wrong: '4*alpha/G = " + str(4*Integer(alpha/G)) + "' does not divide 'gamma = " + str(gamma) + "'!")
        return
    return QuadraticForm(Integer(alpha/G), beta, Integer(gamma/Integer(4*alpha/G)))

#
# Computes an experimental pairing.
#
def get_gouvea_mazur_pseudo_pairing(P, u, v, D):
    if P[2] == 0:
        print("The point 'P' is infinite!")
        return
    if D <= 0:
        print("The value '" + str(-D) + "' is not the discriminant of a positive definite binary quadratic form!")
        return
    if D % 4 == 1 or D % 4 == 2:
        print("The value '" + str(D) + " is not a valid discriminant!")
        return
    if not is_fundamental_discriminant(D):
        print("The discriminant '" + str(-D) + "' is not fundamental!")
        return
    a,b,c = get_elliptic_curve_triple(P[0], P[1])
    if D % 2 == 1:
        print("The discriminant 'D = " + str(D) + " is odd!")
        return
    d = Integer(D/4)
    alpha = v*abs(a*v+c^2*u); G = gcd(v, c)^2
    beta = (b*v^2/G)*inverse_mod(Integer(c^3/G), Integer(alpha/G))
    return QuadraticForm(alpha/G, 2*beta, Integer((beta^2+d)/(alpha/G)))

def get_G(P, Q, D):
    a,b,c = get_elliptic_curve_triple(P[0], P[1]); u,v,w = get_elliptic_curve_triple(Q[0], Q[1])
    A = a*w^2; B = b*w^3; U = c^2*u; V = c^3*v
    alpha = abs(A-U); G = gcd(alpha, V^2); H = gcd(2*B, V)
    return G

def get_alternative_G(P, Q, d):
    a,b,c = get_elliptic_curve_triple(P[0], P[1]); u,v,w = get_elliptic_curve_triple(Q[0], Q[1])
    A = -d*a*w^2; B = -d*b*w^3; U = c^2*u; V = c^3*v
    alpha = abs(A-U); G = gcd(alpha, B^2); H = gcd(B, V)
    return G

In [6]:
##################################
# Computation - General Pairings #
##################################
'''
def list_pairing_orders(E, P, t0, t1)
def generate_multiples_int_points(E, n)
def close_class_subgroup(forms, tuples=True)
def find_id_preimage(E, D, pts, Q)
def find_integer_torsion_point(E)
def get_c(E, r = None): from Griffin-Ono-Tsai
'''
#
# Returns a list of all the pairings generated by the specified elliptic curve 'E' and base point 'P' using values between 't0' and 't1'.
# (THIS IS FOR D_E(t).)
#
def list_pairing_orders(E, P, t0, t1):
    pairings = []
    for t in range(t0, t1+1):
        d_et = get_de_t(E,t)
        if d_et > 0 and is_fundamental_discriminant(d_et):
            Q = [-t, 1]
            pairing = get_elliptic_curve_pairing(P, Q, d_et)
            CG = ClassGroup(d_et)
            x = ClassElement(CG, pairing)
            n = x.get_order()
            pairings += [(pairing, n)]
    return pairings


def generate_multiples_int_points(E, n):
    '''
    Input:
        E: an elliptic curve
        n: positive integer
    Output:
        rational_pts: set of rational points of the form j*P + k*Q
            where j, k <= n and P, Q are integer points of E
    '''
    rational_pts = set()
    for P in E.integral_points():
        for Q in E.integral_points():
            for j in range(1, n+1):
                for k in range(1, n+1):
                    new_point = j*P+k*Q
                    if new_point[1]^2 == new_point[0]^3 + E.a4()*new_point[0] + E.a6():
                        rational_pts.add(new_point)
    return rational_pts

#
# Given a list of quadratic forms, use the group law (composition) to find the rest of the
# group/subgroup generated by these.
#
def close_class_subgroup(forms, tuples=True):
    '''
    input:
        forms: list of quadratic forms represented as 3-tuples if tuples is True, set of ClassGroup objects if tuples is false
    output:
        list of ClassGroup objects representing elements of the class group that we can reach from composing forms
    '''
    if tuples:
        forms = [QuadraticForm(form[0],form[1], form[2]) for form in forms]
    allforms = forms
    for form1 in forms:
        for form2 in forms:
            new_form = form1+form2
            if new_form not in allforms:
                allforms += [new_form]
    if len(allforms) > len(forms):
        return close_class_subgroup(allforms, tuples = False)
    return allforms


def find_id_preimage(E, D, pts, Q):
    '''
    Input:
        E: elliptic curve
        D > 0: discriminant -D divisible by 4
        pts: list or set of points P (tuple or actual point)
        Q: (u, v) on quadratic twist
        
    Output: set of points P that map to identity quad form x^2 + D/4 y^2
    '''
    preim = set()
    identity = (1, 0, Integer(D/4))
    print(identity)
    for point in pts:
        P = (point[0], point[1])
        F = get_elliptic_curve_pairing(P, Q, D)
        if F == identity:
            preim.add(P)
    
    return preim

#
# Takes in any elliptic curve E from database and a torsion point and outputs integral curve 
# in Weierstrass form with same rank and torsion, together with the corresponding torsion point
# (unfortunately, the coeffs tend to be LARGE)
#
def find_integer_torsion_point(E):
    '''
    Input:
        E: elliptic curve, integer coeffs with a point of finite order
    Output: [F, torsion_generator]
        F: integral elliptic curve, takes the form [0, 0, 0, a, b] for integers a, b (Weierstrass form)
        torsion_generator: point (A, B) that generates the torsion of F
    '''
    R.<x,y> = QQ[]
    cubic = -y^2 - E.a1()*x*y - E.a3()*y + x^3 + E.a2()*x^2 + E.a4()*x + E.a6()
    coeffs = WeierstrassForm(cubic)
    weierstrass_form = EllipticCurve([0, 0, 0, Rational(coeffs[0]), Rational(coeffs[1])])
    G = weierstrass_form.torsion_subgroup()
    Q = weierstrass_form(G.gens()[0])
    Q = Q.xy()
    A, B, C = get_rational_num_denom(Q)
    a = weierstrass_form.a4()*C^4
    b = weierstrass_form.a6()*C^6
    F = EllipticCurve([0,0,0,a,b])
    torsion_generator = (A, B)
    return [F, torsion_generator]

#
# Computes constants c(E) from Griffin-Ono-Tsai
#
def get_c(E, r = None):
    '''
    Input
        E: elliptic object
        r: E's rank
    Output
        c(E) from new G-O-T paper
    '''
    if r is None:
        r = E.rank()
    
    omega_r = pi^(r/2) / gamma(r/2 + 1)
    E_tor = E.torsion_order()
    R_Q = E.regulator()
    print(omega_r, E_tor, R_Q)
    
    c = E_tor* omega_r / (2^(r+1) * R_Q^(1.0/2))
    return c

In [12]:
###############################
# Computation -- New Pairings #
###############################
def generate_special_elliptic_curve_data_griffin_ono(torsion_order, t_range, w_range, max_curves):
    curves = []; rank = 28
    while len(curves) < max_curves and rank > 0:
        new_curves = elliptic_curves.rank(rank, n=max_curves-len(curves), tors=torsion_order)
        curves += [curve.integral_short_weierstrass_model() for curve in new_curves]
        rank -= 1
    for E in curves:
        a4 = E.a4(); a6 = E.a6()
        torsion_points = E.torsion_points()
        P = None
        for point in torsion_points:
            if point.order() == torsion_order:
                P = point
                break
        if P == None:
            continue
        d_set = set()
        for t in t_range:
            for w in w_range:
                d = t^3 + a4*t*w^4 - a6*w^6
                if d <= 0 or not is_fundamental_discriminant(4*d):
                    continue
                d_set.add(d)
                Q = (-Rational(t/w^2), Rational(1/w^3), 1)
                form = get_griffin_ono_pseudo_pairing(P, Q, 4*d); form_order = form.order()
                good = form_order == torsion_order
                if not good:
                    print("(a4, a6) = " + str((a4, a6)) + ", P = " + str(P) + ", t = " + str(t) + ", form = " + str(form) + ", order = " + str(form_order))
        print("d_set = " + str(d_set))

def generate_special_elliptic_curve_data_gouvea_mazur(torsion_order, u_range, v_range, max_curves):
    curves = []; rank = 28
    while len(curves) < max_curves and rank > 0:
        new_curves = elliptic_curves.rank(rank, n=max_curves-len(curves), tors=torsion_order)
        curves += [curve.integral_short_weierstrass_model() for curve in new_curves]
        rank -= 1
    for E in curves:
        a4 = E.a4(); a6 = E.a6()
        torsion_points = E.torsion_points()
        P = None
        for point in torsion_points:
            if point.order() == torsion_order:
                P = point
                break
        if P == None:
            continue
        for u in u_range:
            for v in v_range:
                d = v*(u^3 + a4*u*v^2 - a6*v^3)
                if d <= 0 or not is_fundamental_discriminant(4*d):
                    continue
                form = get_gouvea_mazur_pseudo_pairing(P, u, v, 4*d); form_order = form.order()
                good = torsion_order == form_order
                if not good:
                    print("(a4, a6) = " + str((a4, a6)) + ", P = " + str(P) + ", form = " + str(form) + ", order = " + str(form_order))

def generate_general_elliptic_curve_data_griffin_ono(t_range, w_range, max_curves):
    curves = []; rank = 4
    while len(curves) < max_curves and rank > 0:
        new_curves = elliptic_curves.rank(rank, n=max_curves-len(curves))
        curves += [curve.integral_short_weierstrass_model() for curve in new_curves]
        rank -= 1
    for E in curves:
        a4 = E.a4(); a6 = E.a6()
        lower, upper, points = E.simon_two_descent()
        if len(points) == 0:
            continue
        for t in t_range:
            for w in w_range:
                d = t^3 + a4*t*w^4 - a6*w^6
                if d <= 0 or not is_fundamental_discriminant(4*d):
                    continue
                form1 = get_griffin_ono_pseudo_pairing(points[0], (-t/w^2, 1/w^3, 1), 4*d); form_order1 = form1.order()
                for i in range(1, 10):
                    P = i*points[0]
                    form = get_griffin_ono_pseudo_pairing(P, (-t/w^2, 1/w^3, 1), 4*d); form_order = form.order()
                    g = form_order1/gcd(i, form_order1)
                    if True: #form_order != g:
                        print("(a4, a6) = " + str((a4, a6)) + ", order = " + str(form_order) + ", g = " + str(g))
                    
def generate_general_elliptic_curve_data_gouvea_mazur(u_range, v_range, max_curves):
    curves = []; rank = 4
    while len(curves) < max_curves and rank > 0:
        new_curves = elliptic_curves.rank(rank, n=max_curves-len(curves))
        curves += [curve.integral_short_weierstrass_model() for curve in new_curves]
        rank -= 1
    for E in curves:
        a4 = E.a4(); a6 = E.a6()
        lower, upper, points = E.simon_two_descent()
        if len(points) <= 2:
            continue
        for u in u_range:
            for v in v_range:
                d = v*(u^3 + a4*u*v^2 - a6*v^3)
                if d <= 0 or not is_fundamental_discriminant(4*d):
                    continue
                form1 = get_gouvea_mazur_pseudo_pairing(points[0], u, v, 4*d)
                form2 = get_gouvea_mazur_pseudo_pairing(points[1], u, v, 4*d)
                form3 = get_gouvea_mazur_pseudo_pairing(points[2], u, v, 4*d)
                print("(u, v) = " + str((u, v)))
                for i in range(1, 8):
                    for j in range(1, 8):
                        for k in range(1, 8):
                            P = i*points[0] + j*points[1] + k*points[2]
                            form = get_gouvea_mazur_pseudo_pairing(P, u, v, 4*d)
                            expected_form = i*form1 + j*form2 + k*form3
                            if form != expected_form:
                                print("form = " + str(form) + ", expected_form = " + str(expected_form))

def generate_bhargava_data_gouvea_mazur(u_range, v_range, max_curves):
    curves = []; rank = 4
    while len(curves) < max_curves and rank > 0:
        new_curves = elliptic_curves.rank(rank, n=max_curves-len(curves))
        curves += [curve.integral_short_weierstrass_model() for curve in new_curves]
        rank -= 1
    for E in curves:
        a4 = E.a4(); a6 = E.a6()
        lower, upper, points = E.simon_two_descent()
        if len(points) <= 1:
            continue
        print("(a4, a6) = " + str((a4, a6)))
        P1 = 7*points[0]
        for P2 in points:
            P2 = 5*P2
            P3 = -(P1+P2)
            if P1 == P2:
                m = (3*P1[0]^2+a4)/(2*P1[1])
            else:
                m = (P2[1]-P1[1])/(P2[0]-P1[0])
            n = P1[1]-m*P1[0]
            A1, B1, C1 = get_elliptic_curve_triple(P1[0], P1[1])
            A2, B2, C2 = get_elliptic_curve_triple(P2[0], P2[1])
            A3, B3, C3 = get_elliptic_curve_triple(P3[0], P3[1])
            M = C1*C2*C3*m; N = C1*C2*C3*n
            for u in u_range:
                for v in v_range:
                    d = v*(u^3 + a4*u*v^2 - a6*v^3)
                    if d <= 0 or not is_fundamental_discriminant(4*d):
                        continue
                    b = N*v-M*u
                    a1 = A1*v+C1^2*u; a2 = A2*v+C2^2*u; a3 = A3*v+C3^2*u
                    g1 = gcd(C1, v); g2 = gcd(C2, v); g3 = gcd(C3, v)
                    mu1 = inverse_mod(Integer(C1^3/g1^2), Integer(v*a1/g1^2))
                    mu2 = inverse_mod(Integer(C2^3/g2^2), Integer(v*a2/g2^2))
                    mu3 = inverse_mod(Integer(C3^3/g3^2), Integer(v*a3/g3^2))
                    el1 = Integer((g1^2 - C1^3*mu1)/(a1*v))
                    el2 = Integer((g2^2 - C2^3*mu2)/(a2*v))
                    el3 = Integer((g3^2 - C3^3*mu3)/(a3*v))
                    C = C1*C2*C3
                    q1 = g1*(v*b-Integer(B1*v^2/g1^2)*mu1*C)/(v*a1)
                    q2 = g2*(v*b-Integer(B2*v^2/g2^2)*mu2*C)/(v*a2)
                    q3 = g3*(v*b-Integer(B3*v^2/g3^2)*mu3*C)/(v*a3)
                    #if gcd(C1^2*C2^2*C3^2/(g1^2*g2^2*g3^2), v^3*a1*a2*a3/(g1^2*g2^2*g3^2)) != 1:
                    #    print(gcd(C1^2*C2^2*C3^2/(g1^2*g2^2*g3^2), v^3*a1*a2*a3/(g1^2*g2^2*g3^2)))
                    Bg1 = [[[v*C1*C2*C3/(g1*g2*g3), (-v^3*a3*B3*C1*C2*el3 + v^2*B3*C1*C2*g3^2 - v*C3^2*g3^2*b)/(a3*C3^2*g1*g2*g3)], [(-v^3*a2*B2*C1*C3*el2 + v^2*B2*C1*C3*g2^2 - v*C2^2*g2^2*b)/(a2*C2^2*g1*g2*g3), (v^5*a2*a3*B2*B3*C1^2*C2*C3*el2*el3 - v^4*a2*B2*B3*C1^2*C2*C3*g3^2*el2 - v^4*a3*B2*B3*C1^2*C2*C3*g2^2*el3 + v^3*B2*B3*C1^2*C2*C3*g2^2*g3^2 + v^3*a2*B2*C1*C3^3*g3^2*b*el2 + v^3*a3*B3*C1*C2^3*g2^2*b*el3 - v^2*B3*C1*C2^3*g2^2*g3^2*b - v^2*B2*C1*C3^3*g2^2*g3^2*b - a1*a2*a3*C2^2*C3^2*g2^2*g3^2 + v*C2^2*C3^2*g2^2*g3^2*b^2)/(a2*a3*C1*C2^3*C3^3*g1*g2*g3)]], [[(-v^3*a1*B1*C2*C3*el1 + v^2*B1*C2*C3*g1^2 - v*C1^2*g1^2*b)/(a1*C1^2*g1*g2*g3), (v^5*a1*a3*B1*B3*C1*C2^2*C3*el1*el3 - v^4*a1*B1*B3*C1*C2^2*C3*g3^2*el1 - v^4*a3*B1*B3*C1*C2^2*C3*g1^2*el3 + v^3*B1*B3*C1*C2^2*C3*g1^2*g3^2 + v^3*a1*B1*C2*C3^3*g3^2*b*el1 + v^3*a3*B3*C1^3*C2*g1^2*b*el3 - v^2*B3*C1^3*C2*g1^2*g3^2*b - v^2*B1*C2*C3^3*g1^2*g3^2*b - a1*a2*a3*C1^2*C3^2*g1^2*g3^2 + v*C1^2*C3^2*g1^2*g3^2*b^2)/(a1*a3*C1^3*C2*C3^3*g1*g2*g3)], [(v^5*a1*a2*B1*B2*C1*C2*C3^2*el1*el2 - v^4*a1*B1*B2*C1*C2*C3^2*g2^2*el1 - v^4*a2*B1*B2*C1*C2*C3^2*g1^2*el2 + v^3*B1*B2*C1*C2*C3^2*g1^2*g2^2 + v^3*a1*B1*C2^3*C3*g2^2*b*el1 + v^3*a2*B2*C1^3*C3*g1^2*b*el2 - v^2*B2*C1^3*C3*g1^2*g2^2*b - v^2*B1*C2^3*C3*g1^2*g2^2*b - a1*a2*a3*C1^2*C2^2*g1^2*g2^2 + v*C1^2*C2^2*g1^2*g2^2*b^2)/(a1*a2*C1^3*C2^3*C3*g1*g2*g3), (-v^7*a1*a2*a3*B1*B2*B3*C1^2*C2^2*C3^2*el1*el2*el3 + v^6*a1*a2*B1*B2*B3*C1^2*C2^2*C3^2*g3^2*el1*el2 + v^6*a1*a3*B1*B2*B3*C1^2*C2^2*C3^2*g2^2*el1*el3 + v^6*a2*a3*B1*B2*B3*C1^2*C2^2*C3^2*g1^2*el2*el3 - v^5*a1*B1*B2*B3*C1^2*C2^2*C3^2*g2^2*g3^2*el1 - v^5*a2*B1*B2*B3*C1^2*C2^2*C3^2*g1^2*g3^2*el2 - v^5*a1*a2*B1*B2*C1*C2*C3^4*g3^2*b*el1*el2 - v^5*a3*B1*B2*B3*C1^2*C2^2*C3^2*g1^2*g2^2*el3 - v^5*a1*a3*B1*B3*C1*C2^4*C3*g2^2*b*el1*el3 - v^5*a2*a3*B2*B3*C1^4*C2*C3*g1^2*b*el2*el3 + v^4*B1*B2*B3*C1^2*C2^2*C3^2*g1^2*g2^2*g3^2 + v^4*a1*B1*B3*C1*C2^4*C3*g2^2*g3^2*b*el1 + v^4*a1*B1*B2*C1*C2*C3^4*g2^2*g3^2*b*el1 + v^4*a2*B2*B3*C1^4*C2*C3*g1^2*g3^2*b*el2 + v^4*a2*B1*B2*C1*C2*C3^4*g1^2*g3^2*b*el2 + v^4*a3*B2*B3*C1^4*C2*C3*g1^2*g2^2*b*el3 + v^4*a3*B1*B3*C1*C2^4*C3*g1^2*g2^2*b*el3 - v^3*B2*B3*C1^4*C2*C3*g1^2*g2^2*g3^2*b - v^3*B1*B3*C1*C2^4*C3*g1^2*g2^2*g3^2*b - v^3*B1*B2*C1*C2*C3^4*g1^2*g2^2*g3^2*b + v^2*a1^2*a2*a3*B1*C2^3*C3^3*g2^2*g3^2*el1 - v^3*a1*B1*C2^3*C3^3*g2^2*g3^2*b^2*el1 + v^2*a1*a2^2*a3*B2*C1^3*C3^3*g1^2*g3^2*el2 - v^3*a2*B2*C1^3*C3^3*g1^2*g3^2*b^2*el2 + v^2*a1*a2*a3^2*B3*C1^3*C2^3*g1^2*g2^2*el3 - v^3*a3*B3*C1^3*C2^3*g1^2*g2^2*b^2*el3 - v*a1*a2*a3*B3*C1^3*C2^3*g1^2*g2^2*g3^2 - v*a1*a2*a3*B2*C1^3*C3^3*g1^2*g2^2*g3^2 - v*a1*a2*a3*B1*C2^3*C3^3*g1^2*g2^2*g3^2 + v^2*B3*C1^3*C2^3*g1^2*g2^2*g3^2*b^2 + v^2*B2*C1^3*C3^3*g1^2*g2^2*g3^2*b^2 + v^2*B1*C2^3*C3^3*g1^2*g2^2*g3^2*b^2 + a1*a2*a3*C1^2*C2^2*C3^2*g1^2*g2^2*g3^2*b - v*C1^2*C2^2*C3^2*g1^2*g2^2*g3^2*b^3)/(a1*a2*a3*C1^4*C2^4*C3^4*g1*g2*g3)]]]
                    Bg2 = [[[v*C1*C2*C3/(g1*g2*g3), (v^2*B3*C1*C2*g3 - v*C3^2*g3*b)/(a3*C3^2*g1*g2)], [(v^2*B2*C1*C3*g2 - v*C2^2*g2*b)/(a2*C2^2*g1*g3), (v^3*B2*B3*C1^2*C2*C3*g2*g3 - v^2*B3*C1*C2^3*g2*g3*b - v^2*B2*C1*C3^3*g2*g3*b - a1*a2*a3*C2^2*C3^2*g2*g3 + v*C2^2*C3^2*g2*g3*b^2)/(a2*a3*C1*C2^3*C3^3*g1)]], [[(v^2*B1*C2*C3*g1 - v*C1^2*g1*b)/(a1*C1^2*g2*g3), (v^3*B1*B3*C1*C2^2*C3*g1*g3 - v^2*B3*C1^3*C2*g1*g3*b - v^2*B1*C2*C3^3*g1*g3*b - a1*a2*a3*C1^2*C3^2*g1*g3 + v*C1^2*C3^2*g1*g3*b^2)/(a1*a3*C1^3*C2*C3^3*g2)], [(v^3*B1*B2*C1*C2*C3^2*g1*g2 - v^2*B2*C1^3*C3*g1*g2*b - v^2*B1*C2^3*C3*g1*g2*b - a1*a2*a3*C1^2*C2^2*g1*g2 + v*C1^2*C2^2*g1*g2*b^2)/(a1*a2*C1^3*C2^3*C3*g3), (v^4*B1*B2*B3*C1^2*C2^2*C3^2*g1*g2*g3 - v^3*B2*B3*C1^4*C2*C3*g1*g2*g3*b - v^3*B1*B3*C1*C2^4*C3*g1*g2*g3*b - v^3*B1*B2*C1*C2*C3^4*g1*g2*g3*b - v*a1*a2*a3*B3*C1^3*C2^3*g1*g2*g3 - v*a1*a2*a3*B2*C1^3*C3^3*g1*g2*g3 - v*a1*a2*a3*B1*C2^3*C3^3*g1*g2*g3 + v^2*B3*C1^3*C2^3*g1*g2*g3*b^2 + v^2*B2*C1^3*C3^3*g1*g2*g3*b^2 + v^2*B1*C2^3*C3^3*g1*g2*g3*b^2 + a1*a2*a3*C1^2*C2^2*C3^2*g1*g2*g3*b - v*C1^2*C2^2*C3^2*g1*g2*g3*b^3)/(a1*a2*a3*C1^4*C2^4*C3^4)]]]
                    if (Bg1[0][0][1] - Bg2[0][0][1]) % (B3*C1*C2*v^3/(C3^2*g1*g2*g3)).numerator() != 0:
                        print((Bg1[0][0][1] - Bg2[0][0][1]) % (B3*C1*C2*v^3/(C3^2*g1*g2*g3)).numerator())
                    
generate_general_elliptic_curve_data_gouvea_mazur(range(5, 100), range(5, 100), 100)
generate_bhargava_data_gouvea_mazur(range(1, 100), range(1, 100), 100)

(u, v) = (89, 5)
(u, v) = (93, 5)
(u, v) = (97, 5)
(u, v) = (78, 5)
(u, v) = (82, 5)
(u, v) = (86, 5)
(u, v) = (94, 5)
(u, v) = (95, 6)
(u, v) = (97, 6)
(u, v) = (98, 5)
(u, v) = (68, 5)
(u, v) = (72, 5)
(u, v) = (76, 5)
(u, v) = (79, 6)


KeyboardInterrupt: 

In [8]:
##################################
# Converting to Weierstrass Form #
##################################

R.<a1,a3, x,y,z> = QQ[]
cubic = y^2 + a1*x*y + a3*y - x^3
WeierstrassForm(cubic, variables=[x,y,z])

(-1/48*a1^4 + 1/2*a1*a3, 1/864*a1^6 - 1/24*a1^3*a3 + 1/4*a3^2)

In [10]:
#####
# Is it really a pairing? NO?
#####

D = 4*465
E = EllipticCurve([-11, -6])
F = EllipticCurve([-11*Integer(-D/4)^2, -6*Integer(-D/4)^3])
P = E.gens()[0]
Q = F.gens()[0]

for j in range(1, 10):
    R = j*Q
    print("j =",j)
    for i in range(1, 30):
        P_coord = i*P
        Q_coord = (R[0]/Integer(-D/4), R[1]/Integer(-D/4)^2, 1)
        print(i, get_griffin_ono_pseudo_pairing(P_coord, Q_coord, D))

j = 1
1 (7, 4, 67)
2 (2, 2, 233)
3 (7, -4, 67)
4 (5, 0, 93)
5 (7, 4, 67)
6 (2, 2, 233)
7 (7, -4, 67)
8 (5, 0, 93)
9 (7, 4, 67)
10 (2, 2, 233)
11 (7, -4, 67)
12 (15, 0, 31)
13 (7, 4, 67)
14 (2, 2, 233)
15 (7, -4, 67)
16 (5, 0, 93)
17 (7, 4, 67)
18 (23, 16, 23)
19 (7, -4, 67)
20 (5, 0, 93)
21 (7, 4, 67)
22 (2, 2, 233)
23 (7, -4, 67)
24 (15, 0, 31)
25 (7, 4, 67)
26 (2, 2, 233)
27 (7, -4, 67)
28 (5, 0, 93)
29 (7, 4, 67)
j = 2
1 (5, 0, 93)
2 (1, 0, 465)
3 (5, 0, 93)
4 (1, 0, 465)
5 (5, 0, 93)
6 (1, 0, 465)
7 (5, 0, 93)
8 (1, 0, 465)
9 (5, 0, 93)
10 (1, 0, 465)
11 (5, 0, 93)
12 (3, 0, 155)
13 (5, 0, 93)
14 (1, 0, 465)
15 (5, 0, 93)
16 (1, 0, 465)
17 (5, 0, 93)
18 (15, 0, 31)
19 (5, 0, 93)
20 (1, 0, 465)
21 (5, 0, 93)
22 (1, 0, 465)
23 (5, 0, 93)
24 (3, 0, 155)
25 (5, 0, 93)
26 (1, 0, 465)
27 (5, 0, 93)
28 (1, 0, 465)
29 (5, 0, 93)
j = 3
1 (7, -4, 67)
2 (2, 2, 233)
3 (7, 4, 67)
4 (5, 0, 93)
5 (7, -4, 67)
6 (2, 2, 233)
7 (7, 4, 67)
8 (5, 0, 93)
9 (7, -4, 67)
10 (2, 2, 233)
11 (7, 4, 67)
12 (5,

/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:69: RuntimeWarning: cypari2 leaked 1058088 bytes on the PARI stack


KeyboardInterrupt: 

In [11]:
####
# Is it really a pairing (part 2)? Examine torsion point. YES??
####

D = 4*26
a4 = -11; a6 = -6
E = EllipticCurve([a4, a6])
F = EllipticCurve([a4*Integer(-D/4)^2, a6*Integer(-D/4)^3])
P = E.gens()[0]

for i in range(1, 20):
    print(get_griffin_ono_pseudo_pairing(i*P, (-4, 1, 1), D))

(3, -2, 9)
(3, 2, 9)
(1, 0, 26)
(3, -2, 9)
(3, 2, 9)
(1, 0, 26)
(3, -2, 9)
(3, 2, 9)
(1, 0, 26)
(3, -2, 9)
(3, 2, 9)
(1, 0, 26)
(3, -2, 9)
(3, 2, 9)
(1, 0, 26)
(3, -2, 9)
(3, 2, 9)
(1, 0, 26)
(3, -2, 9)
